In [1]:
import numpy as np
import gensim
import nltk
import re
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import seaborn as sns
import pandas as pd
from nltk.tokenize import sent_tokenize
import multiprocessing
import os
from pathlib import Path
import io
from gensim.models import Phrases
from textblob import TextBlob
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from pandas import DataFrame

import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F
import time

In [3]:
input_text = """Bloomington Normal is almost laughably cliché for a midwestern city. Vast swathes of corn envelop winding roads and the heady smell of BBQ smoke pervades the countryside every summer. Yet, underlying the trite norms of Normal is the prescriptive force of tradition—the expectation to fulfill my role as a female Filipino by playing Debussy in the yearly piano festival and enrolling in multivariable calculus instead of political philosophy.So when I discovered the technical demand of bebop, the triplet groove, and the intricacies of chordal harmony after ten years of grueling classical piano, I was fascinated by the music's novelty. Jazz guitar was not only evocative and creative, but also strangely liberating. I began to explore different pedagogical methods, transcribe solos from the greats, and experiment with various approaches until my own unique sound began to develop. And, although I did not know what would be the 'best' route for me to follow as a musician, the freedom to forge whatever path I felt was right seemed to be exactly what I needed; there were no expectations for me to continue in any particular way—only the way that suited my own desires.While journeying this trail, I found myself at Interlochen Arts Camp the summer before my junior year. Never before had I been immersed in an environment so conducive to musical growth: I was surrounded by people intensely passionate about pursuing all kinds of art with no regard for ideas of what art 'should' be. I knew immediately that this would be a perfect opportunity to cultivate my sound, unbounded by the limits of confining tradition. On the first day of camp, I found that my peer guitarist in big band was another Filipino girl from Illinois. Until that moment, my endeavors in jazz guitar had been a solitary effort; I had no one with whom to collaborate and no one against whom I could compare myself, much less someone from a background mirroring my own. I was eager to play with her, but while I quickly recognized a slew of differences between us—different heights, guitars, and even playing styles—others seemed to have trouble making that distinction during performances. Some even went as far as calling me 'other-Francesca.' Thus, amidst the glittering lakes and musky pine needles of Interlochen, I once again confronted Bloomington's frustrating expectations.After being mistaken for her several times, I could not help but view Francesca as a standard of what the 'female Filipino jazz guitarist' should embody. Her improvisatory language, comping style and even personal qualities loomed above me as something I had to live up to. Nevertheless, as Francesca and I continued to play together, it was not long before we connected through our creative pursuit. In time, I learned to draw inspiration from her instead of feeling pressured to follow whatever precedent I thought she set. I found that I grew because of, rather than in spite of, her presence; I could find solace in our similarities and even a sense of comfort in an unfamiliar environment without being trapped by expectation. Though the pressure to conform was still present—and will likely remain present in my life no matter what genre I'm playing or what pursuits I engage in—I learned to eschew its corrosive influence and enjoy the rewards that it brings. While my encounter with Francesca at first sparked a feeling of pressure to conform in a setting where I never thought I would feel its presence, it also carried the warmth of finding someone with whom I could connect. Like the admittedly trite conditions of my hometown, the resemblances between us provided comfort to me through their familiarity. I ultimately found that I can embrace this warmth while still rejecting the pressure to succumb to expectations, and that, in the careful balance between these elements, I can grow in a way that feels both like discove"""

In [23]:
essay_input_corpus = str(input_text) #문장입력
essay_input_corpus = essay_input_corpus.lower()#소문자 변환
essay_input_corpus

"bloomington normal is almost laughably cliché for a midwestern city. vast swathes of corn envelop winding roads and the heady smell of bbq smoke pervades the countryside every summer. yet, underlying the trite norms of normal is the prescriptive force of tradition—the expectation to fulfill my role as a female filipino by playing debussy in the yearly piano festival and enrolling in multivariable calculus instead of political philosophy.so when i discovered the technical demand of bebop, the triplet groove, and the intricacies of chordal harmony after ten years of grueling classical piano, i was fascinated by the music's novelty. jazz guitar was not only evocative and creative, but also strangely liberating. i began to explore different pedagogical methods, transcribe solos from the greats, and experiment with various approaches until my own unique sound began to develop. and, although i did not know what would be the 'best' route for me to follow as a musician, the freedom to forge w

In [ ]:

sentences  = sent_tokenize(essay_input_corpus) #문장 토큰화

In [13]:
split_sentences = []
for sentence in sentences:
    processed = re.sub("[^a-zA-Z]"," ", sentence)
    words = processed.split()
    split_sentences.append(words)


In [14]:
split_sentences 

[['bloomington',
  'normal',
  'is',
  'almost',
  'laughably',
  'clich',
  'for',
  'a',
  'midwestern',
  'city'],
 ['vast',
  'swathes',
  'of',
  'corn',
  'envelop',
  'winding',
  'roads',
  'and',
  'the',
  'heady',
  'smell',
  'of',
  'bbq',
  'smoke',
  'pervades',
  'the',
  'countryside',
  'every',
  'summer'],
 ['yet',
  'underlying',
  'the',
  'trite',
  'norms',
  'of',
  'normal',
  'is',
  'the',
  'prescriptive',
  'force',
  'of',
  'tradition',
  'the',
  'expectation',
  'to',
  'fulfill',
  'my',
  'role',
  'as',
  'a',
  'female',
  'filipino',
  'by',
  'playing',
  'debussy',
  'in',
  'the',
  'yearly',
  'piano',
  'festival',
  'and',
  'enrolling',
  'in',
  'multivariable',
  'calculus',
  'instead',
  'of',
  'political',
  'philosophy',
  'so',
  'when',
  'i',
  'discovered',
  'the',
  'technical',
  'demand',
  'of',
  'bebop',
  'the',
  'triplet',
  'groove',
  'and',
  'the',
  'intricacies',
  'of',
  'chordal',
  'harmony',
  'after',
  'ten

In [16]:
skip_gram = 1
workers = multiprocessing.cpu_count()
workers

6

In [17]:
bigram_transformer = Phrases(split_sentences)
bigram_transformer

In [19]:
model = gensim.models.word2vec.Word2Vec(bigram_transformer[split_sentences], workers=workers, sg=skip_gram, min_count=1)
model

In [20]:
model.train(split_sentences, total_examples=sum([len(sentence) for sentence in sentences]), epochs=100)

(45681, 65600)

In [21]:
#캐릭터 표현하는 단어들을 리스트에 넣어서 필터로 만들고
character_list = ['i', 'my', 'me', 'mine', 'you', 'your', 'they','them',
                'yours', 'he','him','his' 'she','her','it','someone','their', 'myself', 'aunt',
                'brother','cousin','daughter','grandchild','granddaughter','granddson','grandfather',
                'grandmother','great-grandchild','husband','ex-husband','son-in-law', 'daughter-in-law','mother',
                'niece','nephew','parents','sister','son','stepfather','stepmother','stepdaughter', 'stepson',
                'twin','uncle','widow','widower','wife','ex-wife','aunt',
                'baby', 'beget', 'brother', 'buddy', 'conserve', 'counterpart', 'cousin', 'daughter', 'duplicate', 'ex',
                'forefather', 'founder', 'gemini', 'grandchild', 'granddaughter', 'grandfather', 'grandma', 'he', 'helium',
                'husband', 'i', 'in', 'iodine', 'law', 'maine', 'match', 'mine', 'mother', 'nephew', 'niece', 'one', 'parent', 'person',
                'rear', 'sister', 'son', 'stepdaughter', 'stepfather', 'stepmother', 'stepson', 'twin', 'uncle', 'widow', 'widower', 'wife']


In [22]:
####문장에 char_list의 단어들이 있는지 확인하고, 있다면 유사단어를 추출한다.
#우선 토큰화한다.
retokenize = RegexpTokenizer("[\w]+") #줄바꿈 제거하여 한줄로 만들고
token_input_text = retokenize.tokenize(essay_input_corpus)
#print (token_input_text) #토큰화 처리 확인.. 토큰들이 리스트에 담김
token_input_text

['bloomington',
 'normal',
 'is',
 'almost',
 'laughably',
 'cliché',
 'for',
 'a',
 'midwestern',
 'city',
 'vast',
 'swathes',
 'of',
 'corn',
 'envelop',
 'winding',
 'roads',
 'and',
 'the',
 'heady',
 'smell',
 'of',
 'bbq',
 'smoke',
 'pervades',
 'the',
 'countryside',
 'every',
 'summer',
 'yet',
 'underlying',
 'the',
 'trite',
 'norms',
 'of',
 'normal',
 'is',
 'the',
 'prescriptive',
 'force',
 'of',
 'tradition',
 'the',
 'expectation',
 'to',
 'fulfill',
 'my',
 'role',
 'as',
 'a',
 'female',
 'filipino',
 'by',
 'playing',
 'debussy',
 'in',
 'the',
 'yearly',
 'piano',
 'festival',
 'and',
 'enrolling',
 'in',
 'multivariable',
 'calculus',
 'instead',
 'of',
 'political',
 'philosophy',
 'so',
 'when',
 'i',
 'discovered',
 'the',
 'technical',
 'demand',
 'of',
 'bebop',
 'the',
 'triplet',
 'groove',
 'and',
 'the',
 'intricacies',
 'of',
 'chordal',
 'harmony',
 'after',
 'ten',
 'years',
 'of',
 'grueling',
 'classical',
 'piano',
 'i',
 'was',
 'fascinated',
 'by

In [24]:
#리트스로 정리된 개별 토큰을 char_list와 비교해서 존재하는 것만 추출한다.
filtered_chr_text = []
for k in token_input_text:
    for j in character_list:
        if k == j:
            filtered_chr_text.append(j)

filtered_chr_text # 유사단어 비교 추출 완료, 겹치는 단어는 제거하자.
        

['my',
 'in',
 'in',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'my',
 'i',
 'i',
 'me',
 'i',
 'i',
 'i',
 'i',
 'me',
 'in',
 'my',
 'i',
 'i',
 'myself',
 'my',
 'i',
 'i',
 'in',
 'i',
 'i',
 'i',
 'i',
 'my',
 'i',
 'i',
 'my',
 'in',
 'my',
 'in',
 'i',
 'i',
 'one',
 'one',
 'i',
 'i',
 'myself',
 'someone',
 'my',
 'i',
 'i',
 'her',
 'i',
 'i',
 'me',
 'i',
 'i',
 'her',
 'i',
 'i',
 'her',
 'me',
 'i',
 'i',
 'i',
 'i',
 'it',
 'in',
 'i',
 'i',
 'her',
 'i',
 'i',
 'i',
 'i',
 'i',
 'i',
 'in',
 'her',
 'i',
 'i',
 'in',
 'in',
 'in',
 'my',
 'i',
 'i',
 'i',
 'i',
 'in',
 'i',
 'i',
 'it',
 'my',
 'in',
 'i',
 'i',
 'i',
 'i',
 'it',
 'someone',
 'i',
 'i',
 'my',
 'me',
 'their',
 'i',
 'i',
 'i',
 'i',
 'in',
 'i',
 'i',
 'in']

In [25]:
filtered_chr_text_ = set(filtered_chr_text) #중복제거
filtered_chr_text_

{'her', 'i', 'in', 'it', 'me', 'my', 'myself', 'one', 'someone', 'their'}

In [27]:
filtered_chr_text__ = list(filtered_chr_text_) #다시 리스트로 변환
filtered_chr_text__

['my', 'in', 'someone', 'myself', 'i', 'one', 'it', 'me', 'their', 'her']

In [29]:
# 유사단어는 추출했지만 이 코드에서는 별 의미가 없음

sim_words_list = []
for i in filtered_chr_text__:
    ext_sim_words_key = model.most_similar_cosmul(i) #모델적용
    sim_words_list.append(ext_sim_words_key)

/Users/kimkwangil/opt/anaconda3/envs/py37pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `most_similar_cosmul` (Method will be removed in 4.0.0, use self.wv.most_similar_cosmul() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [30]:


sim_words_list

[[('day', 0.9616014361381531),
  ('junior', 0.9609616994857788),
  ('until', 0.9608635902404785),
  ('own', 0.9607545733451843),
  ('journeying', 0.958457887172699),
  ('peer', 0.9575454592704773),
  ('trail', 0.9567568898200989),
  ('desires', 0.9567185044288635),
  ('year', 0.9538546204566956),
  ('arts', 0.9530025124549866)],
 [('careful', 0.9555462002754211),
  ('balance', 0.952503502368927),
  ('elements', 0.9514352083206177),
  ('these', 0.9512788653373718),
  ('grew', 0.9503592848777771),
  ('because', 0.9502040147781372),
  ('grow', 0.9489148259162903),
  ('rather', 0.947279155254364),
  ('succumb', 0.9462975859642029),
  ('discove', 0.9447668194770813)],
 [('much', 0.9898207187652588),
  ('less', 0.9884090423583984),
  ('finding', 0.9823468923568726),
  ('compare', 0.9796267747879028),
  ('connect', 0.9792996644973755),
  ('could', 0.9766407012939453),
  ('myself', 0.9658474326133728),
  ('background', 0.9633925557136536),
  ('against', 0.960185706615448),
  ('with', 0.9577611